In [56]:
import pyw_hnswlib as hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer
import threading
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import re
import string
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display
import imgkit
import os
import warnings
warnings.filterwarnings(action="ignore")

In [57]:
hindi_range = range(int("0900", 16), int("097F", 16))
english_range = range(int("0061", 16), int("007A", 16) + 1)
punjabi_range = range(int("0A00", 16), int("0A7F", 16))
gujarati_range = range(int("0A80", 16), int("0AFF", 16))
telugu_range = range(int("0C00", 16), int("0C7F", 16))
tamil_range = range(int("0B80", 16), int("0BFF", 16))
kannada_range = range(int("0C80", 16), int("0CFF", 16))
odia_range = range(int("0B00", 16), int("0B7F", 16))
bengali_range = range(int("0980", 16), int("09FF", 16))

# define a function to detect the language of a sentence
def detect_language(sentence):
    # count the number of characters in each language range
    hindi_count = sum(1 for c in sentence if ord(c) in hindi_range)
    english_count = sum(1 for c in sentence if ord(c) in english_range)
    punjabi_count = sum(1 for c in sentence if ord(c) in punjabi_range)
    gujarati_count = sum(1 for c in sentence if ord(c) in gujarati_range)
    telugu_count = sum(1 for c in sentence if ord(c) in telugu_range)
    tamil_count = sum(1 for c in sentence if ord(c) in tamil_range)
    kannada_count = sum(1 for c in sentence if ord(c) in kannada_range)
    odia_count = sum(1 for c in sentence if ord(c) in odia_range)
    bengali_count = sum(1 for c in sentence if ord(c) in bengali_range)

    # determine the language with the highest character count
    language_counts = {
        "Hindi": hindi_count,
        "English": english_count,
        "Punjabi": punjabi_count,
        "Gujarati": gujarati_count,
        "Telugu": telugu_count,
        "Tamil": tamil_count,
        "Kannada": kannada_count,
        "Odia": odia_count,
        "Bengali": bengali_count
    }
    max_count = max(language_counts.values())
    language = [k for k, v in language_counts.items() if v == max_count][0]

    return language



In [83]:
index_file_path = "/home/installer/ps/long_term/my_index"


# Load index from binary file
index = hnswlib.Index(space='l2', dim=768)
index.load_index(index_file_path)
index.set_ef(10000)


In [59]:
import sys
size = sys.getsizeof(index)
size

48

In [60]:

# device = "cpu"
# from sentence_transformers import SentenceTransformer

model_salesken = SentenceTransformer("salesken/similarity-eng-hin_latin")


In [61]:
# # Load the model from a pickle file
# with open('model_salesken.pkl', 'rb') as f:
#     model_salesken = pickle.load(f)

In [62]:

# # Save the model to a pickle file
# with open('model_salesken.pkl', 'wb') as f:
#     pickle.dump(model_salesken, f)

In [63]:
# r = pd.read_csv('data/chunk0.csv',  header = None, nrows = 1, low_memory=False)

In [64]:
# Load the pandas dataframe
df1 = pd.read_csv('data/chunk0.csv', usecols= [0, 21],  header = None, low_memory=False)
df2 = pd.read_csv('data/chunk1.csv',  usecols= [0, 21], header = None, low_memory=False)
df3 = pd.read_csv('data/chunk2.csv',  usecols= [0, 21], header = None, low_memory=False)
df4 = pd.read_csv('data/chunk3.csv',  usecols= [0, 21], nrows = 800000, header = None, low_memory=False)
df = pd.concat([df1, df2, df3, df4])




In [65]:

# df = df[[0,21]]
df.columns = ['reg_no', 'subject_content']

In [66]:
df.reset_index(drop=True, inplace = True)
df

,reg_no,subject_content
0,-- Se/E/2015/00001,my grievance i am uploading in pdf format
1,-- Se/E/2015/00002,This is in regards to Vendor Registration Appl...
2,--SEL/E/2015/00001,hjjkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...
3,--SEL/E/2015/00002,"Respected Sir, SUB :Telangana State - ANTI COR..."
4,--SEL/E/2015/00003,This complaint is regarding FIR no. 347 filled...
...,...,...
3799998,PMOPG/D/2016/0263471,FRANK
3799999,PMOPG/D/2016/0263477,NR/RAHU
3800000,PMOPG/D/2016/0263479,PG/SAT
3800001,PMOPG/D/2016/0263483,FRANK


In [67]:
with open("df.pkl", "wb") as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
# df.subject_content.to_csv("tmp.csv")

In [69]:

# Function to split text into lines with maximum words_per_line words per line
def split_text_to_lines(text, words_per_line=20):
    words = text.split()
    lines = []
    line = []
    for word in words:
        line.append(word)
        if len(line) == words_per_line:
            lines.append(' '.join(line))
            line = []
    if line:
        lines.append(' '.join(line))
    return '\n'.join(lines)



In [70]:
with open('final_wording.pkl', 'rb') as f:
    final_wording = pickle.load(f)

In [71]:

def clean_text(text):
    """
    Preprocess textual data by performing the following steps:
    1. Remove punctuation
    2. Convert text to lowercase
    3. Remove digits and special characters
    4. Remove extra whitespaces
    """
    if not isinstance(text, (str, bytes)):
        return text
    
    # remove links
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove digits and special characters
    text = re.sub(r'\d+', '', text)
    
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Correct Roman words using the predefined dictionary
    words = text.split()
    for i in range(len(words)):
        if words[i] in final_wording:
            words[i] = final_wording[words[i]]
    text = ' '.join(words)
    
    return text

In [82]:
# index.set_num_threads(8)

In [73]:
# create a lock
lock = threading.Lock()

# function for performing KNN search
def knn_search(index, query_vec, kk):
    lock.acquire() # acquire the lock before accessing the index
    labels, distances = index.knn_query(query_vec, k=kk)
    lock.release() # release the lock after the search is complete
    return labels, distances

In [124]:
def process_input_string(query):
    query_embedding = model_salesken.encode(clean_text(query))
    labels, distances = knn_search(index, query_embedding, kk=100)
    out  = df[df.reg_no.isin(labels[0])]
    print(len(out))
    out["subject_content_cleaned"] = out.subject_content.apply(clean_text)
    
        # remove empty sentences or sentences with just spaces
    out = out.dropna(subset=["subject_content_cleaned"])
    out = out[out["subject_content_cleaned"].str.strip().astype(bool)]
    out.reset_index(drop=True, inplace = True)
    out["Language"] = out['subject_content_cleaned'].apply(detect_language)
    print(query, str(out.Language.value_counts()))
    print()
    out.drop(["subject_content_cleaned"], axis=1, inplace=True)

    

    for col in out.columns:
        out[col] = out[col].apply(split_text_to_lines)
    # Add distances as a new column
    out['distances'] = distances[0]
    
    # out = out[out.distances < 10.0]
    
    # group the dataframe by language
    grouped = out.groupby('Language')

    # create a new empty dataframe to store the modified output
    new_out = pd.DataFrame()

    # iterate over each language group
    for lang, group in grouped:
        # get the first two rows of the group
        sample = group.head(3)
        # add the sample to the new dataframe
        new_out = pd.concat([new_out, sample])

    # reset the index of the new dataframe
    new_out.reset_index(drop=True, inplace=True)
    

    # replace the original dataframe with the modified one
    out = new_out
    
    # convert the dataframe to an HTML table
    html_table = out.to_html()

    # use imgkit to convert the HTML table to an image
    imgkit.from_string(html_table, "sample_images/"+query+".png")
    # out.to_csv(, index=False, lineterminator='\r\n\n')
    # return out


    
    # return out
    
    

In [75]:
poverty_issues = ['Lack of access to basic necessities such as food and shelter.',
                  'Poor nutrition and health outcomes for impoverished individuals.',
                  'Limited economic opportunities for impoverished individuals and communities.',
                  'Rural poverty and lack of development in many areas.',
                  'High levels of poverty among marginalized and disadvantaged groups.',
                  'Lack of social safety net and government support for impoverished individuals.',
                  'Challenges in accessing education and healthcare for impoverished individuals.',
                  'Child poverty and exploitation, including child labor and trafficking.',
                  'Homelessness and displacement due to poverty and urbanization.',
                  'Persistent poverty in certain regions despite overall economic growth.']


In [76]:
issues = ['High youth unemployment despite economic growth.',
          'Poor quality education, especially in rural areas.',
          'Inadequate healthcare access and system.',
          'Women face discrimination and violence.',
          'Widespread corruption negatively impacts economy and society.',
          'Air and water pollution harm health.',
          'Investment needed in roads, railways, airports.',
          'Social discrimination based on caste persists.',
          'Tensions from communal violence between religious groups.',
          'Large population living in poverty.',
          'Security threats from terrorist organizations.',
          'Increasingly sophisticated cyber threats pose challenges.',
          'Wealth inequality among population.',
          'Children forced into dangerous work.',
          'Struggle for rights and protections.',
          'Reform needed for efficiency, transparency, accessibility.',
          'Challenges in housing, infrastructure, and social integration.',
          'Lack of access to digital technologies limits participation.']


In [100]:
for i in poverty_issues:
    process_input_string(i)
    # break

100
Lack of access to basic necessities such as food and shelter. Language
English    76
Hindi      24
Name: count, dtype: int64

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before 

In [80]:
TOKENIZERS_PARALLELISM = False

In [90]:
final_wording["traficing"]

'trafficking'

In [120]:
df['subject_content'].sample(n=10000).to_csv("allg.csv", index = False)

In [125]:

process_input_string("provident fund PF or gratuity advance not received")



100
provident fund PF or gratuity advance not received Language
English    100
Name: count, dtype: int64

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- 